In [1]:
import turicreate as tc
import pandas as pd
import numpy as np

In [2]:
mw = tc.load_sframe('../binarydata/funcs-encoded')
mw.remove_column('fcount', inplace=True)

apk,function
5868,124151
38,205926
3965,412019
4227,455698
8365,89728
16611,673605
17213,507119
33,204076
497,342467
8526,204076


In [4]:
apks = mw['apk'].unique()
k = apks.shape[0]

In [5]:
sim_recom = tc.item_similarity_recommender.create(mw, user_id='function', item_id='apk', similarity_type='jaccard', 
                                                  only_top_k=k, threshold=0.0, verbose=False)

In [6]:
items = sim_recom.get_similar_items(apks, k=k)

In [8]:
items.head()

apk,similar,score,rank
21855,804,0.44979554414749146,1
21855,3977,0.42904841899871826,2
21855,9141,0.3600617051124573,3
21855,16802,0.22550606727600098,4
21855,3610,0.20764803886413574,5
21855,24303,0.20415878295898438,6
21855,22561,0.18196511268615723,7
21855,2422,0.1654706597328186,8
21855,27588,0.1622059941291809,9
21855,2903,0.16173487901687622,10


In [7]:
a = 8200
b = 11365

In [9]:
def d(a, b, items):
    return items[(items['apk']==a)&(items['similar']==b)]

In [11]:
d(a, b, items)

apk,similar,score,rank
8200,11365,0.1666666865348816,1800


In [13]:
rel = mw[(mw['apk']==a)|(mw['apk']==b)]

In [14]:
rel.materialize()

In [18]:
rel

apk,function
8200,178583
11365,207973
11365,101925
11365,178583
8200,101925
11365,342566
11365,65996


In [19]:
a_funcs = set(rel[rel['apk']==a]['function'])
b_funcs = set(rel[rel['apk']==b]['function'])

In [20]:
a_funcs, b_funcs

({101925, 178583}, {65996, 101925, 178583, 207973, 342566})

In [23]:
len(a_funcs.intersection(b_funcs))/len(a_funcs.union(b_funcs))

0.4

In [24]:
sim_recom_s = tc.item_similarity_recommender.create(rel, user_id='function', item_id='apk', similarity_type='jaccard', 
                                                  only_top_k=k, threshold=0.0, verbose=False)

In [27]:
items2 = sim_recom_s.get_similar_items([a,b], k=k)

In [28]:
items2

apk,similar,score,rank
8200,11365,0.3999999761581421,1
11365,8200,0.3999999761581421,1


In [30]:
i8 = sim_recom.get_similar_items([8200], k=k)

In [31]:
i8[i8['similar']==b]

apk,similar,score,rank
8200,11365,0.1666666865348816,1800


In [35]:
i8[(i8['rank']>1795)&(i8['rank']<1810)]

apk,similar,score,rank
8200,2419,0.1666666865348816,1796
8200,20049,0.1666666865348816,1797
8200,11861,0.1666666865348816,1798
8200,6570,0.1666666865348816,1799
8200,11365,0.1666666865348816,1800
8200,20390,0.1666666865348816,1801
8200,16605,0.1666666865348816,1802
8200,1101,0.1666666865348816,1803
8200,9446,0.1666666865348816,1804
8200,29,0.1666666865348816,1805


In [39]:
#seed_item_set_size = 0 
#nearest_neighbors_interaction_proportion_threshold = 0.9
#degree_approximation_threshold 
sim_recom = tc.item_similarity_recommender.create(mw, user_id='function', item_id='apk', similarity_type='jaccard', 
                                                  only_top_k=k,nearest_neighbors_interaction_proportion_threshold=1, threshold=0.0, verbose=False)
it = sim_recom.get_similar_items([a], k=k)
it[it['similar']==b]

apk,similar,score,rank
8200,11365,0.1666666865348816,1835
